In [90]:
import numpy as np
import bm3d
from pathlib import Path
import pickle
from skimage import io, img_as_float
import nibabel as nib
import os
import glob
from collections import defaultdict

File Paths

In [91]:
icip_data = "D:/VIP Cup/Dataset/ICIP training data/"
denoised_pkl_dataset = "D:/VIP Cup/Output dataset/pkl_data"


Constants

In [92]:
tiff_directories = defaultdict(int)
denoised_tiff_count = 0
original_tiff_count = 0
SIGMA = 0.085
oct_height = 300
oct_width = 300

Functions to check whether a tiff file/directory or not

In [93]:
def is_tiff_dir(curr_path):
    '''function to check if the current directory having any tiff files'''
    num_tiff = len(glob.glob(os.path.join(curr_path, '*.tiff')))
    return num_tiff

def is_tiff(curr_path):
    '''function to check if the current file is a tiff file or not'''
    if os.path.isfile(curr_path) and (".tiff" in curr_path):
        return True
    else:
        return False

Searching tiff directories

In [94]:
tiff_directories = defaultdict(int)
denoised_tiff_count = 0
original_tiff_count = 0
SIGMA = 0.083

dataset = glob.iglob(os.path.join(icip_data, '**'), recursive=True)

for i,file in enumerate(dataset):
    if is_tiff_dir(file):

        current_tiff_count = is_tiff_dir(file)
        tiff_directories[file] = current_tiff_count
        print(f"{len(tiff_directories)}th case {file} ......")
        original_tiff_count += current_tiff_count



print(f"Denoised cases: {len(tiff_directories)}")
#print(f"Denoised tiff images: {denoised_tiff_count}")
print(f"Original tiff images: {original_tiff_count}")


1th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1) ......
2th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (10) ......
3th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (11) ......
4th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (12) ......
5th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13) ......
6th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (14) ......
7th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (15) ......
8th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (16) ......
9th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (17) ......
10th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (18) ......
11th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (19) ......
12th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (2) ......
13th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (20) ......
14th case D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (21) ......
15t

Denoising Function

In [95]:
def bm3d_denoise(noisy_image, npsd_sigma = SIGMA):

    noisy_oct = img_as_float(io.imread(noisy_image, as_gray=True))
    sigma = npsd_sigma
    bm3d_denoised = bm3d.bm3d(noisy_oct, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES)
    return bm3d_denoised

Saving Denoised Image as a .pkl file

In [96]:
def save_denoised_image_as_pkl(denoised_array, output_path):
    
    with open(output_path, 'wb') as file:
        pickle.dump(denoised_array, file)

Output path generating function

In [97]:
def out_path(in_path, output_dir):
    '''generate output path'''
    base_name = os.path.basename(in_path)
    class_name = os.path.basename(os.path.dirname(in_path))

    class_dir = os.path.join(output_dir,class_name)
    os.makedirs(class_dir, exist_ok=True)    

    out_path = class_dir + "/" + base_name + ".pkl"
    return out_path

Open a pickle file

In [98]:
def open_pickle(path_to_pkl):

    with open(path_to_pkl, 'rb') as file:
        loaded_vol_oct = pickle.load(file)

    return loaded_vol_oct

Denoising and Volumetric image conversion loop

In [99]:
for key, value in tiff_directories.items():

    img_dir = key
    num_2d_layers = value
    print(f'{key}: {value}')

    fns = list([str(oct_image) for oct_image in glob.glob(os.path.join(img_dir,'*.tiff*'))])

    #sorting 2d arrays 

    case_no = lambda x: int(x.split('(')[-1].split(')')[0])
    oct_images = sorted(fns, key = lambda y: case_no(y))

    if num_2d_layers == len(oct_images):

        vol_image = np.zeros(shape=(oct_height, oct_width,num_2d_layers))

        for c in range(num_2d_layers):
            
            denoised_image = bm3d_denoise(oct_images[c], SIGMA)

            if  denoised_image.shape == (oct_height,oct_width):

                #print(oct_image)
                #save_out =  os.path.join("D:/VIP Cup/Results/bm3d_one_dir/", os.path.basename(oct_images[c]))
                #save_denoised_image(denoised_image, save_out)

                #denoised_image = np.asarray(denoised_image).astype(np.float64).squeeze()

                if denoised_image.ndim != 2:
                    raise Exception(f'Only 2D data supported. File {denoised_image} has dimension {denoised_image.ndim}.')
                else:
                    vol_image[:,:,c] = denoised_image
            else:
                raise Exception(f"Dimension error: {oct_images[c]}.")
    else:
        raise Exception(f"No of channels available is different: {key}")

    out_name = out_path(img_dir, denoised_pkl_dataset)
    save_denoised_image_as_pkl(vol_image, out_name)

D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1): 52
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (10): 273
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (11): 300
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (12): 258
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (13): 146
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (14): 220
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (15): 87
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (16): 52
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (17): 176
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (18): 300
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (19): 84
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (2): 201
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (20): 153
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (21): 153
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (22): 283
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (23): 283
D:/VIP Cup/Dataset/ICIP training data\0\RawDat

For Testing Purposes...

Denoising and Volumetric image conversion for one directory

In [86]:
curr_dir = list(tiff_directories.keys())[0]
case_1_denoised = None

img_dir = curr_dir
num_2d_layers = list(tiff_directories.values())[0]

#print(f'{key}: {value}')

fns = list([str(oct_image) for oct_image in glob.glob(os.path.join(img_dir,'*.tiff*'))])
#sorting 2d arrays 
case_no = lambda x: int(x.split('(')[-1].split(')')[0])
oct_images = sorted(fns, key = lambda y: case_no(y))

if num_2d_layers == len(oct_images):
    vol_image = np.zeros(shape=(oct_height, oct_width,num_2d_layers))
    for c in range(num_2d_layers):
        
        denoised_image = bm3d_denoise(oct_images[c], SIGMA)
        if  denoised_image.shape == (oct_height,oct_width):
            print(oct_images[c])
            #save_out =  os.path.join("D:/VIP Cup/Results/bm3d_one_dir/", os.path.basename(oct_images[c]))
            #save_denoised_image(denoised_image, save_out)
            #denoised_image = np.asarray(denoised_image).astype(np.float64).squeeze()
            if denoised_image.ndim != 2:
                raise Exception(f'Only 2D data supported. File {denoised_image} has dimension {denoised_image.ndim}.')
            else:
                vol_image[:,:,c] = denoised_image
        else:
            raise Exception(f"Dimension error: {oct_images[c]}.")
else:
    raise Exception(f"No of channels available is different: {key}")
out_name = out_path(img_dir, denoised_pkl_dataset)
#save_denoised_image_as_pkl(vol_image, out_name)
case_1_denoised = vol_image


D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (1).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (2).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (3).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (4).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (5).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (6).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (7).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (8).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (9).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (10).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (11).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (12).tiff
D:/VIP Cup/Dataset/ICIP training data\0\RawDataQA (1)\RawDataQA1 (13).tiff
D:/VIP Cup/Dataset/ICIP training d